# A file to fix the ecal_ang() and ecal_sum() lambda loss term functions
## (1) remove keras and (2) allow for different sizing

In [1]:
import tensorflow as tf
from tensorflow import py_function, float32, Tensor
from tensorflow.keras import backend as K
import math

import numpy as np
import tensorflow as tf
#from PIL import Image
import h5py
from numpy import asarray
#import cv2

In [36]:
# SIMPLE DEBUGGING VARIABLES
#daxis=(1,1)
#c = tf.constant([[1.0, 2.0, 4.0], [3.0, 4.0, 5.0]])
#c

<tf.Tensor 'Sum_5:0' shape=() dtype=float32>

In [3]:
def GetDataAngle(datafile, imgs3dscale =1, imgs3dpower=1, e_pscale = 100, angscale=1, angtype='theta', thresh=1e-4):
    print ('Loading Data from .....', datafile)
    f = h5py.File(datafile,'r')                    # load data into f variable
    ang = np.array(f.get(angtype))                 # ang is an array of angle data from f, one value is concatenated onto the latent vector
    imgs3d = np.array(f.get('ECAL'))* imgs3dscale    # imgs3d is a 3d array, cut from the cylinder that the calorimeter produces -has 25 layers along z-axis
    e_p = np.array(f.get('energy'))/e_pscale       # e_p is an array of scaled energy data from f, one value is concatenated onto the latent vector
    imgs3d[imgs3d < thresh] = 0        # when imgs3d values are less than the threshold, they are reset to 0
    
    # set imgs3d, e_p, and ang as float 32 datatypes
    imgs3d = imgs3d.astype(np.float32)
    e_p = e_p.astype(np.float32)
    ang = ang.astype(np.float32)
    
    imgs3d = np.expand_dims(imgs3d, axis=-1)         # insert a new axis at the beginning for imgs3d
    
    # sum along axis
    ecal = np.sum(imgs3d, axis=(1, 2, 3))    # summed imgs3d data, used for training the discriminator
     
    # imgs3d ^ imgs3dpower
    if imgs3dpower !=1.:
        imgs3d = np.power(imgs3d, imgs3dpower)
            
    # imgs3d=ecal data; e_p=energy data; ecal=summed imgs3d (used to train the discriminator); ang=angle data
    return imgs3d, e_p, ang, ecal

In [4]:
# load and process the image
imgs3d, e_p, ang, ecal = GetDataAngle('Ele_VarAngleMeas_100_200_005.h5')

Loading Data from ..... Ele_VarAngleMeas_100_200_005.h5


In [8]:
image = imgs3d[0, :, :, :]
#inv_image = Lambda(tf.pow, arguments={'a':1./power})(image) #get back original image
power = 1.   # from anglearch discriminator
inv_image = tf.math.pow(image, 1./power)
inv_image.shape
#print(image.tolist())

TensorShape([51, 51, 25, 1])

In [15]:
channel = 'channels_last'    # for cpu
if channel =='channels_last':
    daxis=(1,2,3)
    dshape=(51, 51, 25,1)
else:
    daxis=(2,3,4)
    dshape=(1, 51, 51, 25)

### Original ecal_sum()

In [5]:
# Summming cell energies
def ecal_sum(image, power):
    image = K.pow(image, 1./power)
    sum = K.sum(image, axis=daxis)
    return sum

### New ecal_sum() 
#### do i need to multiply by a value for smaller images??

In [6]:
# Summming cell energies
def ecal_sum(image, power=1.0, channel_included=True, inverted=False):
    # pgan image shape will be either (z=shape/2, x=shape, y=shape) or (channel=1, z=shape/2, x=shape, y=shape)
    if channel_included:    
        image = tf.squeeze(image) # get rid of channel dimension
        
    # switch dimensions ordering from pgan (z,x,y) back to anglegan (x,y,z)
    if size != 51:
        image = np.moveaxis(image, 1, -1)   # move z back
    # the tensor is now [num_images,x,y,z]
    
    if not inverted:
        # pre-process the image (the og image is inverted in getdataangle())
        image = tf.pow(image, 1./power)   
    
    daxis = (1,2,3)   # (x,y,z)
    # sum the values along the daxis
    sum = tf.math.reduce_sum(image, daxis)   
    return sum

In [7]:
# test
#print(tf.Session().run(new_ecal_sum(c, 2)))
print(ecal_sum(inv_image, daxis).shape)
print(ecal_sum(inv_image, daxis))
print(ecal_sum(inv_image, daxis).shape)
print(ecal_sum(inv_image, daxis))

NameError: name 'inv_image' is not defined

### Original ecal_angle()

In [25]:
# Calculating angle from image
def tf1_ecal_angle(image, power):
    print(image.shape)
    image = K.squeeze(image, axis=daxis)# squeeze along channel axis
    print(image.shape)
    # size of ecal
    x_shape= K.int_shape(image)[1]
    y_shape= K.int_shape(image)[2]
    z_shape= K.int_shape(image)[3]
    print('shapes: ',x_shape, y_shape, z_shape)
    sumtot = K.sum(image, axis=(1,2,3))# sum of events
    
    # get 1. where event sum is 0 and 0 elsewhere
    amask = K.tf.where(tf.math.equal(sumtot, 0.0), K.ones_like(sumtot) , K.zeros_like(sumtot))
    masked_events = K.sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = K.sum(K.sum(image, axis=(2,3)) * (K.cast(K.expand_dims(K.arange(x_shape), 0), dtype='float32') + 0.5) , axis=1)# sum for x position * x index
    y_ref = K.sum(K.sum(image, axis=(1,3)) * (K.cast(K.expand_dims(K.arange(y_shape), 0), dtype='float32') + 0.5), axis=1)
    z_ref = K.sum(K.sum(image, axis=(1,2)) * (K.cast(K.expand_dims(K.arange(z_shape), 0), dtype='float32') + 0.5), axis=1)
    x_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(x_ref) , x_ref/sumtot)# return max position if sumtot=0 and divide by sumtot otherwise
    y_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(y_ref) , y_ref/sumtot)
    z_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(z_ref), z_ref/sumtot)
    
    # reshape    
    x_ref = K.expand_dims(x_ref, 1)
    y_ref = K.expand_dims(y_ref, 1)
    z_ref = K.expand_dims(z_ref, 1)

    sumz = K.sum(image, axis =(1,2)) # sum for x,y planes going along z

    # Get 0 where sum along z is 0 and 1 elsewhere
    zmask = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz) , K.ones_like(sumz))
        
    x = K.expand_dims(K.arange(x_shape), 0) # x indexes
    x = K.cast(K.expand_dims(x, 2), dtype='float32') + 0.5
    y = K.expand_dims(K.arange(y_shape), 0)# y indexes
    y = K.cast(K.expand_dims(y, 2), dtype='float32') + 0.5
  
    # barycenter for each z position
    x_mid = K.sum(K.sum(image, axis=2) * x, axis=1)
    y_mid = K.sum(K.sum(image, axis=1) * y, axis=1)
    x_mid = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), x_mid/sumz) # if sum != 0 then divide by sum
    y_mid = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), y_mid/sumz) # if sum != 0 then divide by sum

    # Angle Calculations
    z = (K.cast(K.arange(z_shape), dtype='float32') + 0.5)  * K.ones_like(z_ref) # Make an array of z indexes for all events
    zproj = K.sqrt(K.maximum((x_mid-x_ref)**2.0 + (z - z_ref)**2.0, K.epsilon()))# projection from z axis with stability check
    m = K.tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj), (y_mid-y_ref)/zproj)# to avoid divide by zero for zproj =0
    m = K.tf.where(K.tf.less(z, z_ref),  -1 * m, m)# sign inversion
    ang = (math.pi/2.0) - tf.atan(m)# angle correction
    zmask = K.tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj) , zmask)
    ang = ang * zmask # place zero where zsum is zero
    
    ang = ang * z  # weighted by position
    sumz_tot = z * zmask # removing indexes with 0 energies or angles

    #zunmasked = K.sum(zmask, axis=1) # used for simple mean 
    #ang = K.sum(ang, axis=1)/zunmasked # Mean does not include positions where zsum=0

    ang = K.sum(ang, axis=1)/K.sum(sumz_tot, axis=1) # sum ( measured * weights)/sum(weights)
    ang = K.tf.where(K.equal(amask, 0.), ang, 100. * K.ones_like(ang)) # Place 100 for measured angle where no energy is deposited in events
    
    ang = K.expand_dims(ang, 1)
    return ang

In [67]:
#tf1_ecal_angle(inv_image, 1.0)

In [8]:
# Calculating angle from image
def tf2_ecal_angle(image, power):
    if K.image_data_format() =='channels_last':
       image = K.squeeze(image, axis=4)
    else:                      # channels first
       image = K.squeeze(image, axis=1)
    image = K.pow(image, 1./power)
    
    # size of ecal
    x_shape= K.int_shape(image)[1]
    y_shape= K.int_shape(image)[2]
    z_shape= K.int_shape(image)[3]
    print('shapes ------- ', x_shape, y_shape, z_shape)
    sumtot = K.sum(image, axis=(1,2,3))# sum of events
    
    # get 1. where event sum is 0 and 0 elsewhere
    amask = tf.where(K.equal(sumtot, 0.0), K.ones_like(sumtot) , K.zeros_like(sumtot))
    masked_events = K.sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = K.sum(K.sum(image, axis=(2,3)) * (K.cast(K.expand_dims(K.arange(x_shape), 0), dtype='float32') + 0.5) , axis=1)# sum for x position * x index
    y_ref = K.sum(K.sum(image, axis=(1,3)) * (K.cast(K.expand_dims(K.arange(y_shape), 0), dtype='float32') + 0.5), axis=1)
    z_ref = K.sum(K.sum(image, axis=(1,2)) * (K.cast(K.expand_dims(K.arange(z_shape), 0), dtype='float32') + 0.5), axis=1)
    x_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(x_ref) , x_ref/sumtot)# return max position if sumtot=0 and divide by sumtot otherwise
    y_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(y_ref) , y_ref/sumtot)
    z_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(z_ref), z_ref/sumtot)
    
    #reshape    
    x_ref = K.expand_dims(x_ref, 1)
    y_ref = K.expand_dims(y_ref, 1)
    z_ref = K.expand_dims(z_ref, 1)

    sumz = K.sum(image, axis =(1,2)) # sum for x,y planes going along z

    # Get 0 where sum along z is 0 and 1 elsewhere
    zmask = tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz) , K.ones_like(sumz))
        
    x = K.expand_dims(K.arange(x_shape), 0) # x indexes
    x = K.cast(K.expand_dims(x, 2), dtype='float32') + 0.5
    y = K.expand_dims(K.arange(y_shape), 0)# y indexes
    y = K.cast(K.expand_dims(y, 2), dtype='float32') + 0.5
  
    # barycenter for each z position
    x_mid = K.sum(K.sum(image, axis=2) * x, axis=1)
    y_mid = K.sum(K.sum(image, axis=1) * y, axis=1)
    x_mid = tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), x_mid/sumz) # if sum != 0 then divide by sum
    y_mid = tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), y_mid/sumz) # if sum != 0 then divide by sum

    # Angle Calculations
    z = (K.cast(K.arange(z_shape), dtype='float32') + 0.5)  * K.ones_like(z_ref) # Make an array of z indexes for all events
    zproj = K.sqrt(K.maximum((x_mid-x_ref)**2.0 + (z - z_ref)**2.0, K.epsilon()))# projection from z axis with stability check
    m = tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj), (y_mid-y_ref)/zproj)# to avoid divide by zero for zproj =0
    m = tf.where(tf.less(z, z_ref),  -1 * m, m)# sign inversion
    ang = (math.pi/2.0) - tf.atan(m)# angle correction
    zmask = tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj) , zmask)
    ang = ang * zmask # place zero where zsum is zero
    
    ang = ang * z  # weighted by position
    sumz_tot = z * zmask # removing indexes with 0 energies or angles

    #zunmasked = K.sum(zmask, axis=1) # used for simple mean 
    #ang = K.sum(ang, axis=1)/zunmasked # Mean does not include positions where zsum=0

    ang = K.sum(ang, axis=1)/K.sum(sumz_tot, axis=1) # sum ( measured * weights)/sum(weights)
    ang = tf.where(K.equal(amask, 0.), ang, 100. * K.ones_like(ang)) # Place 100 for measured angle where no energy is deposited in events
    
    ang = K.expand_dims(ang, 1)
    return ang

In [9]:
image = imgs3d[0]
tf2_ecal_angle(imgs3d, 1.0)

shapes -------  51 51 25


<tf.Tensor: shape=(5000, 1), dtype=float32, numpy=
array([[1.3089745],
       [1.1618682],
       [1.1150758],
       ...,
       [1.6336285],
       [1.6282631],
       [1.8434817]], dtype=float32)>

### new ecal_angle() -- accounting for different sizes & no keras -- in progress

In [10]:
# Calculating angle from image -- called in conditional lambda layer for the discriminator
def ecal_angle(image, size, power=1.0, channel_included=True, inverted=False):
    # pgan image shape will be either (z=shape/2, x=shape, y=shape) or (channel=1, z=shape/2, x=shape, y=shape)
    if channel_included:    
        image = tf.squeeze(image) # get rid of channel dimension
        
    # switch dimensions ordering from pgan (z,x,y) back to anglegan (x,y,z)
    if size != 51:
        image = np.moveaxis(image, 1, -1)   # move z back
    
    xdim, ydim, zdim = 1,2,3   # the tensor is now [num_images,x,y,z]
    
    if not inverted:
        # pre-process the image (the og image is inverted in getdataangle())
        image = tf.pow(image, 1./power)   
     
    # size of ecal
    x_shape, y_shape, z_shape = int(size), int(size), int(size/2)
    print('shapes ----- ', x_shape, y_shape, z_shape )
    sumtot = tf.math.reduce_sum(image, (1,2,3))# sum of events
    print('sumtot: ', sumtot)
    
    # get 1. where event sum is 0 and 0 elsewhere
    amask = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(sumtot) , tf.zeros_like(sumtot))
    masked_events = tf.math.reduce_sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (2,3)) * (tf.cast(tf.expand_dims(tf.range(x_shape), 0), dtype='float32') + 0.5), axis=1)# sum for x position * x index
    y_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (1,3)) * (tf.cast(tf.expand_dims(tf.range(y_shape), 0), dtype='float32') + 0.5), axis=1)
    z_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (1,2)) * (tf.cast(tf.expand_dims(tf.range(z_shape), 0), dtype='float32') + 0.5), axis=1)
    # return max position if sumtot=0 and divide by sumtot otherwise
    x_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(x_ref), x_ref/sumtot)
    y_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(y_ref), y_ref/sumtot)
    z_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(z_ref), z_ref/sumtot)
    
    # reshape - put in value at the beginning    
    x_ref = tf.expand_dims(x_ref, 1)
    y_ref = tf.expand_dims(y_ref, 1)
    z_ref = tf.expand_dims(z_ref, 1)

    sumz = tf.math.reduce_sum(image, axis =(1,2)) # sum for x,y planes going along z

    # Get 0 where sum along z is 0 and 1 elsewhere
    zmask = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz) , tf.ones_like(sumz))
        
    x = tf.expand_dims(tf.range(x_shape), 0) # x indexes
    x = tf.cast(tf.expand_dims(x, 2), dtype='float32') + 0.5
    y = tf.expand_dims(tf.range(y_shape), 0)# y indexes
    y = tf.cast(tf.expand_dims(y, 2), dtype='float32') + 0.5
  
    # barycenter for each z position
    x_mid = tf.math.reduce_sum(tf.math.reduce_sum(image, axis=2) * x, axis=1)
    y_mid = tf.math.reduce_sum(tf.math.reduce_sum(image, axis=1) * y, axis=1)
    x_mid = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz), x_mid/sumz) # if sum != 0 then divide by sum
    y_mid = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz), y_mid/sumz) # if sum != 0 then divide by sum

    # Angle Calculations
    z = (tf.cast(tf.range(z_shape), dtype='float32') + 0.5)  * tf.ones_like(z_ref) # Make an array of z indexes for all events
    epsilon = 0.0000007  # replaces k.epsilon(), used as fluff value to prevent /0 errors
    zproj = tf.math.sqrt(tf.math.maximum((x_mid-x_ref)**2.0 + (z - z_ref)**2.0, epsilon))# projection from z axis with stability check
    m = tf.where(tf.math.equal(zproj, 0.0), tf.zeros_like(zproj), (y_mid-y_ref)/zproj)# to avoid divide by zero for zproj =0
    m = tf.where(tf.math.less(z, z_ref),  -1 * m, m)   # sign inversion
    ang = (math.pi/2.0) - tf.atan(m)   # angle correction
    zmask = tf.where(tf.math.equal(zproj, 0.0), tf.zeros_like(zproj), zmask)
    ang = ang * zmask # place zero where zsum is zero
    
    ang = ang * z  # weighted by position
    sumz_tot = z * zmask # removing indexes with 0 energies or angles

    #zunmasked = tf.math.reduce_sum(zmask, axis=1) # used for simple mean 
    #ang = tf.math.reduce_sum(ang, axis=1)/zunmasked # Mean does not include positions where zsum=0

    ang = tf.math.reduce_sum(ang, axis=1)/tf.math.reduce_sum(sumz_tot, axis=1) # sum ( measured * weights)/sum(weights)
    ang = tf.where(tf.math.equal(amask, 0.), ang, 100. * tf.ones_like(ang)) # Place 100 for measured angle where no energy is deposited in events
    
    ang = tf.expand_dims(ang, 1)
    return ang

In [16]:
# TESTING
imgs3d.shape #51
new_ecal_angle(imgs3d, 51, 1.0)#, daxis)
imgs3d_64 = resize(imgs3d, 64)
new_ecal_angle(imgs3d_64, 64, 1.0)#, daxis)
#imgs3d_16 = resize(imgs3d, 16)
#print('resized to 16')
#new_ecal_angle(imgs3d_16, 16, 1.0)#, daxis)

shapes -----  51 51 25
sumtot:  tf.Tensor([150.78053 109.73003 192.05147 ... 131.63718 144.31905 197.36601], shape=(5000,), dtype=float32)


MemoryError: Unable to allocate 4.88 GiB for an array with shape (5000, 64, 64, 32) and data type float64

In [15]:
#from PIL import Image
import h5py
from numpy import asarray
#import cv2
def resize(imgs3d, size, mode='rectangle'):
    """Short summary.

    Parameters
    ----------
    imgs3d : type
        Description of parameter `imgs3d`.
    size : type
        Description of parameter `size`.
    mode : type
        Description of parameter `mode`.

    Returns
    -------
    type
        Description of returned object.

    """
    imgs3d = imgs3d[:, :, :, :, 0]    # drop the channels dimension
    nmbr_of_images = len(imgs3d)

    if mode == 'rectangle':
        resized_imgs3d = np.zeros((nmbr_of_images, size, size, int(size/2)))    # create an array to hold all nmbr_of_images resized imgs3d

        for num_img in np.arange(nmbr_of_images):         # index through the nmbr_of_images 3d images packed in
            img3d = imgs3d[num_img, :, :, :]    # grab an individual [51,51,25] 3d image

            # pad centrally with zeroes to [64x64x32], do this step first so that the framing is the same for all images
            resized_img3d = np.pad(img3d, ((7,6), (7,6), (4,3)), mode='minimum')

            if size == 64:   # put in the padded image [64,64,32]
                resized_imgs3d[num_img, :, :, :] = resized_img3d

            else:   # size < 64: we need to zoom out to lower the resolution
                # resize XY-plane to (size x size)
                xy_resized_img3d = np.zeros((size, size, 25))   # create an empty 3d_image to store changes
                for z_index in np.arange(25):    # index through the 25 calorimeter layers of the z-axis
                    img2d = img3d[:, :, z_index]   # grab a 2d image from the xy plane
                    resized_img2d = cv2.resize(img2d, dsize=(size, size), interpolation=cv2.INTER_NEAREST)
                    xy_resized_img3d[:, :, z_index] = resized_img2d   # save our resized_img2d in the img3d corresponding to the calorimeter layer

                # resize YZ-plane to (size x size/2)
                resized_img3d = np.zeros((size, size, int(size/2)))   # create an empty 3d_image to store changes            # resize YZ-plane to (size,size)=square or (size,size/2)=rectangle
                for x_index in np.arange(size):    # index through the x-axis
                    img2d = xy_resized_img3d[x_index, :, :]
                    resized_img2d = cv2.resize(img2d, dsize=(int(size/2), size), interpolation=cv2.INTER_NEAREST)
                    resized_img3d[x_index, :, :] = resized_img2d   # save our resized_img2d in the img3d corresponding to the x layer

            # save the resized 3d image in the matrix holding all nmbr_of_images 3d images
            resized_imgs3d[num_img, :, :, :] = resized_img3d

    elif mode == 'square':
        if size == 64:
            print('ERROR - Square mode is not compatible with size 64! The max size for square mode is 32.')
        else:
            resized_imgs3d = np.zeros((nmbr_of_images, size, size, size)) # create an array to hold all nmbr_of_images resized imgs3d

            for num_img in np.arange(nmbr_of_images):     # index through the nmbr_of_images 3d images packed in
                img3d = imgs3d[num_img, :, :, :]    # grab an individual [51,51,25] 3d image

                img3d = np.pad(img3d, ((0,0), (0,0), (4,3)), mode='minimum') # pad centrally with zeroes to [51x51x32]

                # resize XY-plane to (size x size)
                xy_resized_img3d = np.zeros((size, size, 25))   # create an empty 3d_image to store changes
                for z_index in np.arange(25):    # index through the 25 calorimeter layers of the z-axis
                    img2d = img3d[:, :, z_index]   # grab a 2d image from the xy plane
                    resized_img2d = cv2.resize(img2d, dsize=(size, size), interpolation=cv2.INTER_NEAREST)
                    xy_resized_img3d[:, :, z_index] = resized_img2d   # save our resized_img2d in the img3d corresponding to the calorimeter layer

                # resize YZ-plane to (size x size)
                resized_img3d = np.zeros((size, size, size))   # create an empty 3d_image to store changes
                for x_index in np.arange(size):    # index through the 51 values of x-axis
                    img2d = xy_resized_img3d[x_index, :, :]
                    resized_img2d = cv2.resize(img2d, dsize=(size, size), interpolation=cv2.INTER_NEAREST)
                    resized_img3d[x_index, :, :] = resized_img2d   # save our resized_img2d in the img3d corresponding to the x layer

                # save our 3d image in the matrix holding all nmbr_of_images 3d images
                resized_imgs3d[num_img, :, :, :] = resized_img3d

    # reorganize dimensions: (num_imgs, x,y,z) --> (num_imgs, z,x,y)
    print("before &&&& :", resized_imgs3d.shape)
    resized_imgs3d = np.moveaxis(resized_imgs3d, 3, 1)
    print("after  &&&& :", resized_imgs3d.shape)

    return resized_imgs3d   # returns a [nmbr_of_images, size||size/2, size, size] np.array matrix that is nmbr_of_images 3d images [size||size/2, size, size]

### new ecal_angle() -- converts to og size -- in progress, currently only works with 64-->OG

In [112]:
# Function that takes in the pgan generated 3d images of size [nmbr_of_images, 32, 64, 64] and resizes them to the desired [nmbr_of_images, 51, 51, 25] OG shape
# include channels changes??? -- may need to debug if channels value is included in pgan_imgs
def restore_pics(pgan_imgs3d):
    # matrix dimensions: [nmbr_of_images, 32, 64, 64] --> [nmbr_of_images, 51, 51, 25]
    og_dim_imgs3d = np.moveaxis(pgan_imgs3d, 1, -1)   # move z: [nmbr_of_images, 64, 64, 32]
    og_dim_imgs3d = og_dim_imgs3d[:, 7:58, 7:58, 4:29]   # crop centrally (corresponding to resize()) to og_dimensions 51x51x25
    return og_dim_imgs3d

def restore_pic(image, size):
    # matrix dimensions: [32, 64, 64] --> [51, 51, 25]
    restored_image = np.moveaxis(image, 0, -1)   # move z back: [x,y,z]
    if size == 64:
        restored_image = restored_image[7:58, 7:58, 4:29]   # crop centrally (corresponding to resize()) to og_dimensions 51x51x25
    else: # size = 8, 16, 32
                # resize XY-plane to (size x size)
                xy_restored_image = np.zeros((51,51,size/2))   # create an empty 3d_image to store changes
                for z_index in np.arange(inte(size/2)):    # index through the 25 calorimeter layers of the z-axis
                    img2d = restored_image[:, :, z_index]   # grab a 2d image from the xy plane
                    resized_img2d = cv2.resize(img2d, dsize=(51,51), interpolation=cv2.INTER_NEAREST)
                    xy_restored_image[:, :, z_index] = resized_img2d   # save our resized_img2d in the img3d corresponding to the calorimeter layer

                # resize YZ-plane to (size x size/2)
                restored_image = np.zeros((size, size, int(size/2)))   # create an empty 3d_image to store changes            # resize YZ-plane to (size,size)=square or (size,size/2)=rectangle
                for x_index in np.arange(size):    # index through the x-axis
                    img2d = xy_restored_image[x_index, :, :]
                    resized_img2d = cv2.resize(img2d, dsize=(int(size/2), size), interpolation=cv2.INTER_NEAREST)
                    restored_image[x_index, :, :] = resized_img2d   # save our resized_img2d in the img3d corresponding to the x layer
    return restored_image

In [ ]:
# Calculating angle from image
def ecal_angle(image, size, channel_included=False, inverted=True):#, daxis):
    # pgan image shape will be either (z=shape/2, x=shape, y=shape) or (channel=1, z=shape/2, x=shape, y=shape)
    if channel_included:    
        image = tf.squeeze(image) # get rid of channel dimension
    
    # convert pgan image [32,64,64] to OG [51,51,25] size
    image = restore_pic(image, size)
    xdim, ydim, zdim = 0, 1, 2  
    
    if not inverted:
        # pre-process the image (the og image is inverted in getdataangle())
        image = tf.pow(image, 1./power)   
    
    # size of ecal
    x_shape, y_shape, z_shape = 51, 51, 25
    sumtot = tf.math.reduce_sum(image, (xdim,ydim,zdim))# sum of events

    # get 1. where event sum is 0 and 0 elsewhere
    amask = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(sumtot) , tf.zeros_like(sumtot))
    masked_events = tf.math.reduce_sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (ydim,zdim)) * (tf.cast(tf.expand_dims(tf.range(x_shape), 0), dtype='float32') + 0.5), xdim)# sum for x position * x index
    y_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (xdim,zdim)) * (tf.cast(tf.expand_dims(tf.range(y_shape), 0), dtype='float32') + 0.5), xdim)
    z_ref = tf.math.reduce_sum(tf.math.reduce_sum(image, (xdim,ydim)) * (tf.cast(tf.expand_dims(tf.range(z_shape), 0), dtype='float32') + 0.5), xdim)
    # return max position if sumtot=0 and divide by sumtot otherwise
    x_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(x_ref), x_ref/sumtot)
    y_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(y_ref), y_ref/sumtot)
    z_ref = tf.where(tf.math.equal(sumtot, 0.0), tf.ones_like(z_ref), z_ref/sumtot)
    
    # reshape - put in value at the beginning    
    x_ref = tf.expand_dims(x_ref, 0)
    y_ref = tf.expand_dims(y_ref, 0)
    z_ref = tf.expand_dims(z_ref, 0)

    sumz = tf.math.reduce_sum(image, axis =(xdim,ydim)) # sum for x,y planes going along z

    # Get 0 where sum along z is 0 and 1 elsewhere
    zmask = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz) , tf.ones_like(sumz))
        
    x = tf.expand_dims(tf.range(x_shape), 0) # x indexes
    x = tf.cast(tf.expand_dims(x, 1), dtype='float32') + 0.5
    y = tf.expand_dims(tf.range(y_shape), 0)# y indexes
    y = tf.cast(tf.expand_dims(y, 1), dtype='float32') + 0.5
  
    # barycenter for each z position
    x_mid = tf.math.reduce_sum(tf.math.reduce_sum(image, axis=1) * x, axis=0)
    y_mid = tf.math.reduce_sum(tf.math.reduce_sum(image, axis=0) * y, axis=0)
    x_mid = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz), x_mid/sumz) # if sum != 0 then divide by sum
    y_mid = tf.where(tf.math.equal(sumz, 0.0), tf.zeros_like(sumz), y_mid/sumz) # if sum != 0 then divide by sum

    # Angle Calculations
    z = (tf.cast(tf.range(z_shape), dtype='float32') + 0.5)  * tf.ones_like(z_ref) # Make an array of z indexes for all events
    epsilon = 0.0000007  # replaces k.epsilon(), used as fluff value to prevent /0 errors
    zproj = tf.math.sqrt(tf.math.maximum((x_mid-x_ref)**2.0 + (z - z_ref)**2.0, epsilon))# projection from z axis with stability check
    m = tf.where(tf.math.equal(zproj, 0.0), tf.zeros_like(zproj), (y_mid-y_ref)/zproj)# to avoid divide by zero for zproj =0
    m = tf.where(tf.math.less(z, z_ref),  -1 * m, m)   # sign inversion
    ang = (math.pi/2.0) - tf.atan(m)   # angle correction
    zmask = tf.where(tf.math.equal(zproj, 0.0), tf.zeros_like(zproj), zmask)
    ang = ang * zmask # place zero where zsum is zero
    
    ang = ang * z  # weighted by position
    sumz_tot = z * zmask # removing indexes with 0 energies or angles

    #zunmasked = tf.math.reduce_sum(zmask, axis=1) # used for simple mean 
    #ang = tf.math.reduce_sum(ang, axis=1)/zunmasked # Mean does not include positions where zsum=0

    ang = tf.math.reduce_sum(ang, axis=0)/tf.math.reduce_sum(sumz_tot, axis=0) # sum ( measured * weights)/sum(weights)
    ang = tf.where(tf.math.equal(amask, 0.), ang, 100. * tf.ones_like(ang)) # Place 100 for measured angle where no energy is deposited in events
    
    ang = tf.expand_dims(ang, 0)
    return ang

In [ ]:
# !!!NEEDS WORK!!! prep the images for the fake layer in logistic loss function
def prep_dnn(real_image_input):
    image = Input(shape=dshape) 
    h = Flatten()(image)
    dnn = Model(image, h) # h is flattened x from line 150 in anglegan discriminator = anglearch3dgan.py
    dnn.summary()
    # Output
    dnn_out = dnn(image)
    return dnn_out